In [1]:
# import packages
import pickle
import os
import re
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, f1_score, accuracy_score, roc_auc_score, roc_curve, auc, recall_score

2024-04-25 21:08:16.996456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# create function to import models
def get_exported_models(import_dir):
    """
     This function takes the path to
     the import directory
     for each lagged model
    """
    # initialize empty list to return
    models = []
    # loop over all the files in the import directory
    # we can use the listdir() method on the os class
    # this lists all the files and sub-directories
    for file in os.listdir(import_dir):
        # make sure the file name ends with .pkl
        if file.endswith('.pkl'):
            # get the file name
            # this returns tuple of file name and extension
            # so we pick [0] to get the file name
            file_name = os.path.splitext(file)[0]
            # read the file and append
            # get full file path
            # we need to do this to read from the pickle file
            filepath = os.path.join(import_dir, file)
            # call pickle to extract file
            # we now use rb to read binary since we exported as bin
            with open(filepath, "rb") as f:
                # load the model
                model = pickle.load(f)
            # append to list
            models.append((file_name, model))
    # return the list
    return models

In [21]:
# import all models
#-----------------#

# import logit models
logit_import_dir = "[ENTER PATH FOR LOGIT MODELS]"
# run and extract
logit_models = get_exported_models(logit_import_dir)

# import random forest models
random_forest_import_dir = "[ENTER PATH FOR RANDOM FOREST MODELS]"
# run and extract
random_forest_models = get_exported_models(random_forest_import_dir)

# import the nn models
nn_import_dir = "[ENTER PATH FOR NEURAL NETWORK MODELS]"
# run and extract
nn_models = get_exported_models(nn_import_dir)

# import the svc models
svc_import_dir = "[ENTER PATH FOR SVC MODELS]"
# run and extract
svc_models = get_exported_models(svc_import_dir)

# import gradient boosting models
gradient_boosting_models = "[ENTER PATH FOR XGBOOST MODELS]"
# run and extract
gradient_boosting_models = get_exported_models(gradient_boosting_models)

2024-04-25 21:13:56.431180: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://9139cb17442b43a581bcb423697f09ad: INVALID_ARGUMENT: ram://9139cb17442b43a581bcb423697f09ad is a directory.
2024-04-25 21:13:56.578724: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://ac9e780d0a344311bedd0dba484dca4b: INVALID_ARGUMENT: ram://ac9e780d0a344311bedd0dba484dca4b is a directory.
2024-04-25 21:13:56.730449: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://9e8629867d614772a986fb81d03195d8: INVALID_ARGUMENT: ram://9e8629867d614772a986fb81d03195d8 is a directory.
2024-04-25 21:13:56.874062: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://5e41f4bc0e754319990a7765662dd281: INVALID_ARGUMENT: ram://5e41f4bc0e754319990a7765662dd281 is a directory.
2024-04-25 21:13:57.019434: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://fd1253e7f0b44821b97f855c3c84de18: INVALID_ARGUMENT: ram://fd1253e7f0b44821b97

In [22]:
# import and load data
#--------------------#

# import raw data for testing
data_raw = pd.read_excel('[ENTER PATH FOR THE DATA]', index_col=0)

In [23]:
# set up function to loop over and test different
# thresholds then extract the best one
def find_best_threshold(y_predicted_raw, y_true):
    # create range from 0-1
    # in increments of 0.0001
    threshold_intervals = np.arange(0, 1.0001, 0.0001)
    # now loop over each interval and run the model
    # to see which one performs best
    # create list to store the results
    results = []
    for threshold in threshold_intervals:
        # compare at each threshold
        y_pred_at_threshold = (y_predicted_raw > threshold).astype(int)
        # calculate the f1 score for the predicted
        threshold_f1 = f1_score(y_true, y_pred_at_threshold)
        # create tuple of each threshold and f1
        results.append((threshold_f1, y_pred_at_threshold, threshold))
    # find the threshold with the max f1
    max_threshold_tuple = max(results, key=lambda x: x[0])
    # now extract the threshold from tuple
    best_threshold = max_threshold_tuple[2]
    y_pred = max_threshold_tuple[1]
    best_f1 = max_threshold_tuple[0]
    # return the result
    return [best_threshold, y_pred, best_f1]

In [24]:
# path to store the exported figures
# roc_curve plots
roc_fig_dir_path = '/Users/jean-philippbruehwiler/Desktop/master-thesis-code/model-plots/cm-plots/cm-roc-plots/'

In [25]:
# helper function to create roc_curve
def roc_curve_plot(y_true, y_pred_prob, lag, fig_dir_path, model_name):
    # get the fpr, trp, and thresholds
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    # call method on fpr and tpr
    roc_auc = auc(fpr, tpr)
    
    # create plot
    plt.figure(figsize=(10, 6))
    # we insert the area under curve up to 2 decimal places
    # we can insert using the modulo operator - %
    plt.plot(fpr, tpr, color='lightblue', linewidth=2, label='ROC Curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='darkred', linewidth=2, linestyle=':')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"Logit ROC Curve for {lag} Month Lag")
    plt.legend(loc="lower right")
    file_name = f"roc_curve_{model_name}_lag_{lag}.png"
    save_path = os.path.join(fig_dir_path, file_name)
    plt.savefig(save_path, dpi=300)
    plt.show()

In [26]:
# helper function to flatten list of nested lists
# the predicted values are returned like this:
# i.e. [[0],[1],[0],[1]]
# we need to convert it to this:
# i.e. [0,1,0,1]
def flatten_list(matrix_list):
    flat_list = []
    for row in matrix_list:
        flat_list.append(row[0])
    return flat_list

In [27]:
# set up helper function to extract the correct model
# based on the lag, this takes the list of a certain
# model and returns the model
def extract_correct_lag_model(models, lag):
    """This function take models list
       and the lag to match"""
    # create regex pattern based on lag
    lag_match_pattern = rf"{lag}-month-lag"
    # loop over models
    for model in models:
        # use .search() method to check the entire string
        # check the first part of tuple for name
        # then return second part with the model
        # tuple: (file name of model, model.pkl file)
        if re.search(lag_match_pattern, model[0]):
            return model[1]
    # if no model is found return none
    # this should never be triggered but good
    # to include just in case
    return None

In [28]:
# function to find average for row
def avg_pred_prob(row):
    return row.sum() / len(row)

In [29]:
# set up lags for looping
lags = [3, 6, 9, 12, 18]

In [74]:
# the models are already trained so we just need to test the models
# create function to run
def run_consensus_model(data, lag, test_size):
    """This model accepts testing data and a lag
       it then uses a simple voting concensus to determine
       class, i.e. if a majority of models say 1,
       then the model returns 1"""

    # make a copy of the original DataFrame to avoid modifying it
    data_copy = data.copy()

    # modify dataset for lag
    # we want to set the recession indicator back by the lag so that t0 is aligned with t+lag
    data_copy[f"nber_recession_{lag}_month_lag"] = data_copy['nber_recession'].shift(-lag)

    # drop the original recession column and na values
    data_copy = data_copy.drop(columns=['nber_recession'])
    data_copy = data_copy.dropna()

    # set up training and testing data
    X = data_copy.drop(columns=[f"nber_recession_{lag}_month_lag"])
    y = data_copy[f"nber_recession_{lag}_month_lag"]

    # set up training and testing data
    # we don't need the trainin data in this case, only the testing data
    _, X_test, _, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # extract the relevant model for the given lag
    random_forest = extract_correct_lag_model(random_forest_models, lag)
    svc = extract_correct_lag_model(svc_models, lag)
    gradient_boosting = extract_correct_lag_model(gradient_boosting_models, lag)
    neural_network = extract_correct_lag_model(nn_models, lag)
    logit_model = extract_correct_lag_model(logit_models, lag)

    # make the calculations for each model
    rand_forest_pred = random_forest.predict_proba(X_test)[:,1]
    svc_pred = svc.predict_proba(X_test)[:,1]
    gbc_pred = gradient_boosting.predict_proba(X_test)[:,1]
    logit_pred = logit_model.predict_proba(X_test)[:,1]

    # neural network needs to handled a bit differently
    # we first predict directly, then flatten list
    # of lists which it returns
    nn_predictions_prob_raw = neural_network.predict(X_test)
    # flatten list of lists
    nn_predictions_prob = flatten_list(nn_predictions_prob_raw)

    # create a dataframe of the consensus predictions
    model_probs = pd.DataFrame({
    'rand_forest_pred': rand_forest_pred,
    'svc_pred': svc_pred,
    'gbc_pred': gbc_pred,
    'nn_pred': nn_predictions_prob,
    'logit_pred': logit_pred
    })

    # find the average vote for each row
    # use apply to get the average value for each row
    # we pass in the function then the row is passed as the arg
    model_probs['y_pred_avg'] = model_probs.apply(avg_pred_prob, axis=1)

    # now we can compare the accuracy of the consensus model
    y_pred_raw = model_probs['y_pred_avg'].values

    # extract the data for the best threshold
    model_results = find_best_threshold(y_pred_raw, y_test)
    # extract the relevant results
    threshold = model_results[0]
    y_pred = model_results[1]
    best_f1 = model_results[2]
    
    # create a confusion matrix to visualize results
    conf_mat = confusion_matrix(y_test, y_pred)
    
    # make the roc_curve plot
    # this does not take into account
    # the threshold we ind. calculate
    roc_curve_plot(y_test, y_pred_raw, lag, roc_fig_dir_path, 'cm')

    # create a confusion matrix to visualize results
    conf_mat = confusion_matrix(y_test, y_pred)

    # get predicted values and metrics
    metrics_obj = {
       'accuracy': accuracy_score(y_test, y_pred),
       'precision': precision_score(y_test, y_pred),
       'recall': recall_score(y_test, y_pred),
       'f1': best_f1,
       'roc_auc': roc_auc_score(y_test, y_pred_raw),
       }

    return {'data': data_copy,
            'best_threshold': threshold,
            'y_true': y_test,
            'y_pred': y_pred,
            'y_pred_raw': y_pred_raw,
            'confusion_matrix': conf_mat,
            'model_metrics': metrics_obj}

In [ ]:
# run the model for each lag
consensus_results = [(f"{lag}_month_lag_results", run_consensus_model(data_raw, lag, 0.2)) for lag in lags]

In [76]:
# make a dataframe of all accuracy results
headers_metrics = ['lag', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'conf_matrix', 'threshold']
# store the results for each iteration
iteration_metrics = []
# iterate over results
for result in consensus_results:
    # extract from the tuple
    metrics = result[1]['model_metrics']
    # extract each value
    values = [val for _, val in metrics.items()]
    # insert name of lag
    values.insert(0, result[0])
    # get the confusion matric
    conf_matrix = result[1]['confusion_matrix']
    # append to values
    values.append(conf_matrix)
    # get the best threshold
    threshold = result[1]['best_threshold']
    # append
    values.append(threshold)
    # append to the list
    iteration_metrics.append(values)
# convert to a dataframe
metric_data = pd.DataFrame(iteration_metrics, columns=headers_metrics)

print(metric_data)

                    lag  accuracy  precision    recall        f1   roc_auc  \
0   3_month_lag_results  0.972973   0.923077  0.800000  0.857143  0.987469   
1   6_month_lag_results  0.938776   0.692308  0.947368  0.800000  0.961760   
2   9_month_lag_results  0.931507   0.733333  0.647059  0.687500  0.945280   
3  12_month_lag_results  0.910959   0.666667  0.818182  0.734694  0.929619   
4  18_month_lag_results  0.875862   0.607143  0.708333  0.653846  0.915634   

            conf_matrix  threshold  
0   [[132, 1], [3, 12]]     0.5746  
1   [[120, 8], [1, 18]]     0.4878  
2   [[125, 4], [6, 11]]     0.6074  
3   [[115, 9], [4, 18]]     0.5792  
4  [[110, 11], [7, 17]]     0.4922  


In [17]:
# go through and see if the model is over or underestimating recessions
headers_false_true_summary = ['lag', 'recession_true', 'recession_true_pred', 'recession_false', 'recession_false_pred', 'false_pos_rate', 'false_neg_rate']

# store iteration calculations
iteration_summaries = []

# loop over data
for result in consensus_results:
    # extract the relevant data
    data = result[1]
    y_true_pred = pd.DataFrame({'y_actual': data['y_true'], 'y_predicted': data['y_pred']})

    # create row of data with the calculations
    true_pos = np.sum(y_true_pred['y_actual'] == 1)
    true_neg = np.sum(y_true_pred['y_actual'] == 0)
    pred_pos = np.sum(y_true_pred['y_predicted'] == 1)
    false_pos_rate = np.sum((y_true_pred['y_actual'] == 0) & (y_true_pred['y_predicted'] == 1)) / (np.sum(y_true_pred['y_actual'] == 0))
    false_neg_rate = np.sum((y_true_pred['y_actual'] == 1) & (y_true_pred['y_predicted'] == 0)) / (np.sum(y_true_pred['y_actual'] == 1))

    # create a list of the stats to pass in
    summary_stats = [true_pos, pred_pos, true_neg, len(y_true_pred) - pred_pos, false_pos_rate, false_neg_rate]

    # insert lag name
    summary_stats.insert(0, result[0])

    # append to result list
    iteration_summaries.append(summary_stats)

# convert to df
complete_summary_stats = pd.DataFrame(iteration_summaries, columns=headers_false_true_summary)

# print results
print(complete_summary_stats)

                    lag  recession_true  recession_true_pred  recession_false  \
0   3_month_lag_results              15                   13              133   
1   6_month_lag_results              19                   25              128   
2   9_month_lag_results              17                   15              129   
3  12_month_lag_results              22                   27              124   
4  18_month_lag_results              24                   27              121   

   recession_false_pred  false_pos_rate  false_neg_rate  
0                   135        0.007519        0.200000  
1                   122        0.054688        0.052632  
2                   131        0.031008        0.352941  
3                   119        0.072581        0.181818  
4                   118        0.082645        0.291667  


In [ ]:
# set up writer to export the summary stats
path = '[ADD PATH TO EXPORT RESULTS]'
writer = pd.ExcelWriter(path, engine='openpyxl')

In [ ]:
# export to excel
metric_data.to_excel(writer, sheet_name='summary_stats', index=False)

In [ ]:
# add summary stats to excel output
complete_summary_stats.to_excel(writer, sheet_name='pos_neg_acc_summary', index=False)
# close writer
writer.close()

In [ ]:
# create new writer to export the raw predictions
path_prob_raw = '../raw-pred/cm-pred-raw.xlsx'
writer_raw_prob = pd.ExcelWriter(path_prob_raw, engine='openpyxl')
# loop over all the models and get the raw prediction probabilities
# as well as the y_true data for that loop
for lag, res in zip(lags, consensus_results):
    # get the results
    res_data = res[1]
    # concat into new df
    res_df = pd.DataFrame({'y_true': res_data['y_true'], 'y_pred_raw': res_data['y_pred_raw']})
    # export to new sheet
    res_df.to_excel(writer_raw_prob, sheet_name=f"{lag}_rf", index=False)
# close writer
writer_raw_prob.close()